In [350]:
import numpy as np
from aux import computeEmissions, isRare, getTrigramCount, getBigramCount

In [351]:
#return count(x, y) / count(y)
def getEmission(emissions, x, y):
    if(isRare(emissions, x)):
        return emissions['_RARE_'][y]
    else:
        return emissions[x][y]

In [352]:
def getq(q, u, v, w):
    return q[(u, v, w)]

In [353]:
def getAllWordsAndTags():
    with open('ner_rare.counts') as f_input:
        tags = set()
        words = set()
        for line in f_input:
            tokens = line.strip().split()
            
            if(tokens[1] == 'WORDTAG'):
                tag = tokens[2]
                tags.add(tag)
                word = tokens[3]
                words.add(word)
    return words, tags

#output: T[x] -> set of all tags 'y' s.t. e(x|y) > 0
def computeT(emissions):
    
    T = dict()
    words, tags = getAllWordsAndTags()
    
    for word in words:
        tags_with_positive_emissions = []
        for tag in tags:
            if(getEmission(emissions, word, tag) > 0):
                tags_with_positive_emissions.append(tag)
        T[word] = tags_with_positive_emissions
    
    return T
        
    

In [354]:
#k is given as input assuming x is 1-indexed
def getT(T, x, k, emissions):
    if(k == 0 or k == -1):
        return ['*']
    else:
        if(isRare(emissions, x[k - 1])):
            return T['_RARE_']
        else:
            return T[x[k - 1]]

In [365]:
#input: x[0], x[1], ... x[n - 1]
#     : T[x] -> set of all tags 'y' s.t. e(x|y) > 0

def tagUsingViterbi(x, emissions, T, q):
    
    #initialization
    pi = dict()
    bp = dict()
    
    pi[(0, '*', '*')] = 1
    
    #tagged sequence has same length as the input sequence
    y = x[:]
    
    for k in range(1, len(x) + 1):
        for u in  getT(T, x, k - 1, emissions):
            for v in getT(T, x, k, emissions):
                
                pi[(k, u, v)] = -1
                
                for w in getT(T, x, k - 2, emissions):
                    
                    try:
                        this_probability = pi[(k - 1, w, u)] * getq(q, w, u, v) * getEmission(emissions, x[k - 1], v)
                        
                        if(pi[(k, u, v)]  < this_probability):
                            pi[(k, u, v)] = this_probability
                            bp[(k, u, v)] = w

                    # as of now, exception can be raised by getq only
                    except Exception as e:
                        pass
    
    
    max_prob = -1
    n = len(x)
    for u in getT(T, x, n - 1, emissions):
        for v in getT(T, x, n, emissions):
            try:
                if(max_prob < pi[(n, u, v)]) * getq(q, u, v, 'STOP'):
                    max_prob = pi[(n, u, v)] * getq(q, u, v, 'STOP')
                    y[n - 2] = u
                    y[n - 1] = v
            except Exception as e:
                pass
              

    for k in range(n - 3, -1, -1):
        y[k] = bp[(k + 2 + 1, y[k + 1], y[k + 2])]
        
    return (y, pi)

In [366]:
#this function assumes that y is 1-indexed
def gety(y, k):
    if(k <= 0):
        return '*'
    else:
        return y[k - 1]
def getLogLikelihood(x, y, pi):
    log_likelihood = []
    for k in range(1, len(x) + 1):
        log_prob = np.log(pi[(k, gety(y, k - 1), gety(y, k))])
        log_likelihood.append(log_prob)
    
    return log_likelihood

In [367]:
def writeThisSentence(f_output, x, y, log_likelihood):
    for (word, tag, log_prob) in zip(x, y, log_likelihood):
        this_line = ' '.join((word, tag, '{}'.format(log_prob)))
        this_line = this_line + '\n'
        f_output.write(this_line)
    f_output.write('\n')

In [368]:
def computeq():
    count_of_trigrams = dict()
    count_of_bigrams = dict()
    
    with open('ner_rare.counts') as f_input:
        for line in f_input:
            tokens = line.strip().split()
            if(tokens[1] == '2-GRAM'):
                u, v = tokens[2], tokens[3]
                count_of_bigrams[(u, v)] = int(tokens[0])
            elif(tokens[1] == '3-GRAM'):
                u, v, w = tokens[2], tokens[3], tokens[4]
                count_of_trigrams[(u, v, w)] = int(tokens[0])
    q = dict()
    for (u, v, w) in count_of_trigrams:
        q[(u, v, w)] = float(count_of_trigrams[(u, v, w)])/ float(count_of_bigrams[(u, v)])
        
    return q

In [369]:
#initial parameters
emissions = computeEmissions()
T = computeT(emissions)
q = computeq()

with open('ner_dev.dat') as f_input, open('5_2.txt', 'w') as f_output:
    x = []
    for line in f_input:
        tokens = line.strip().split()
        
        if(len(tokens) > 0):
            x.append(tokens[0])
        else:
            #x has buffered a sentence
            y, pi = tagUsingViterbi(x, emissions, T, q)
            log_likelihood = getLogLikelihood(x, y, pi)
            
            writeThisSentence(f_output, x, y, log_likelihood)
            
            x = []